# The Validation Set Approach

In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt

In this section, we'll explore the use of the validation set approach in order to estimate the
test error rates that result from fitting various linear models on the ${\tt Auto}$ data set.

In [2]:
df1 = pd.read_csv('datasets/Auto.csv', na_values='?').dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 27.7+ KB


We begin by using the ${\tt sample()}$ function to split the set of observations
into two halves, by selecting a random subset of 196 observations out of
the original 392 observations. We refer to these observations as the training
set.

We'll use the ${\tt random\_state}$ parameter in order to set a seed for
${\tt python}$’s random number generator, so that you'll obtain precisely the same results each time. It is generally a good idea to set a random seed when performing an analysis such as cross-validation
that contains an element of randomness, so that the results obtained can be reproduced precisely at a later time.

In [4]:
train_df = df1.sample(196, random_state = 1)
test_df = df1[~df1.isin(train_df)].dropna(how = 'all')

X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

We then use ${\tt LinearRegression()}$ to fit a linear regression to predict ${\tt mpg}$ from ${\tt horsepower}$ using only
the observations corresponding to the training set.

In [6]:
lm = skl_lm.LinearRegression()
#YOUR CODE
reg = lm.fit(X_train, y_train)
print(f'score: {reg.score(X_test, y_test)}, coef: {reg.coef_}, intercept: {reg.intercept_}')

score: 0.5878396082171262, coef: [-0.15964606], intercept: 40.3338030434159


We now use the ${\tt predict()}$ function to estimate the response for the test
observations, and we use ${\tt sklearn}$ to caclulate the MSE.

In [12]:
#YOUR CODE
from sklearn.metrics import mean_squared_error
y_pred = lm.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mse

23.361902892587224

Therefore, the estimated test MSE for the linear regression fit is 23.36. We
can use the ${\tt PolynomialFeatures()}$ function to estimate the test error for the polynomial
and cubic regressions.

In [20]:
#YOUR CODE
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2).fit(X_train)
X_train_poly = poly.transform(X_train)
y_pred_poly = lm.fit(X_train_poly, y_train).predict(poly.transform(X_test))
mse_poly_2 = mean_squared_error(y_test, y_pred_poly)

poly = PolynomialFeatures(3).fit(X_train)
X_train_poly = poly.transform(X_train)
y_pred_poly = lm.fit(X_train_poly, y_train).predict(poly.transform(X_test))
mse_poly_3 = mean_squared_error(y_test, y_pred_poly)

print(f'mse_2: {mse_poly_2}, mse_3: {mse_poly_3}')

mse_2: 20.252690858346554, mse_3: 20.32560936597247


These error rates are 20.25 and 20.33, respectively. If we choose a different
training set instead, then we will obtain somewhat different errors on the
validation set. We can test this out by setting a different random seed:

In [24]:
train_df = df1.sample(196, random_state = 2)
test_df = df1[~df1.isin(train_df)].dropna(how = 'all')

X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

#YOUR CODE
reg = lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

poly = PolynomialFeatures(2).fit(X_train)
X_train_poly = poly.transform(X_train)
y_pred_poly = lm.fit(X_train_poly, y_train).predict(poly.transform(X_test))
mse_poly_2 = mean_squared_error(y_test, y_pred_poly)

poly = PolynomialFeatures(3).fit(X_train)
X_train_poly = poly.transform(X_train)
y_pred_poly = lm.fit(X_train_poly, y_train).predict(poly.transform(X_test))
mse_poly_3 = mean_squared_error(y_test, y_pred_poly)

print(f'mse: {mse}, mse_2: {mse_poly_2}, mse_3: {mse_poly_3}')

mse: 25.10853905288967, mse_2: 19.722533470492426, mse_3: 19.92136786007268


Using this split of the observations into a training set and a validation
set, we find that the validation set error rates for the models with linear,
quadratic, and cubic terms are 25.11, 19.72, and 19.92, respectively.

These results are consistent with our previous findings: a model that
predicts ${\tt mpg}$ using a quadratic function of ${\tt horsepower}$ performs better than
a model that involves only a linear function of ${\tt horsepower}$, and there is
little evidence in favor of a model that uses a cubic function of ${\tt horsepower}$.

# Leave-One-Out Cross-Validation

The LOOCV estimate can be automatically computed for any generalized linear model using the `LeaveOneOut()` and `KFold()` functions.

In [51]:
#YOUR CODE
# determine MSE and STD for a given number of folds (.eg 392)
from sklearn.model_selection import KFold
kf = KFold(n_splits=392, shuffle=True, random_state=1)
mse_values = []
for train_index, test_index in kf.split(df1):
    X_train = df1.iloc[train_index]['horsepower'].values.reshape(-1,1)
    y_train = df1.iloc[train_index]['mpg']
    X_test = df1.iloc[test_index]['horsepower'].values.reshape(-1,1)
    y_test = df1.iloc[test_index]['mpg']
    reg = lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    mse_values.append(mean_squared_error(y_test, y_pred))
mse_mean = np.mean(mse_values)
mse_std = np.std(mse_values) 
print(f'mse: {mse_mean}, std: {mse_std}')

mse: 24.231513517929226, std: 36.79731503640535


Our cross-validation estimate for the test error is approximately 24.23. We can repeat this procedure for increasingly complex polynomial fits.
To automate the process, we use the `for()` function to initiate a for loop
which iteratively fits polynomial regressions for polynomials of order `i = 1`
to `i = 5` and computes the associated cross-validation error. 


In [39]:
#YOUR CODE
# determine MSE and STD for polynomial of different degrees
for i in range(1, 6):
    mse_values = []
    for train_index, test_index in kf.split(df1):
        X_train = df1.iloc[train_index]['horsepower'].values.reshape(-1,1)
        y_train = df1.iloc[train_index]['mpg']
        X_test = df1.iloc[test_index]['horsepower'].values.reshape(-1,1)
        y_test = df1.iloc[test_index]['mpg']
        poly = PolynomialFeatures(i).fit(X_train)
        X_train_poly = poly.transform(X_train)
        y_pred_poly = lm.fit(X_train_poly, y_train).predict(poly.transform(X_test))
        mse_values.append(mean_squared_error(y_test, y_pred_poly))
    mse_mean = np.mean(mse_values)
    mse_std = np.std(mse_values) 
    print(f'poly: {i} - mse: {mse_mean}, std: {mse_std}')

poly: 1 - mse: 24.23151351792922, std: 36.79731503640535
poly: 2 - mse: 19.24821312448968, std: 34.998446151782325
poly: 3 - mse: 19.33498406412144, std: 35.765135677973824
poly: 4 - mse: 19.424430308539886, std: 35.683352757212276
poly: 5 - mse: 19.033216707145417, std: 35.31733192537356


Here we see a sharp drop in the estimated test MSE between
the linear and quadratic fits, but then no clear improvement from using
higher-order polynomials.

# k-Fold Cross-Validation

The `KFold` function can (intuitively) also be used to implement `k`-fold CV. Below we
use `k = 10`, a common choice for `k`, on the `Auto` data set. We once again set
a random seed and initialize a vector in which we will print the CV errors
corresponding to the polynomial fits of orders one to ten.

In [48]:
#YOUR CODE
# determine MSE and STD for polynomial of different degrees

from sklearn.model_selection import cross_val_score
cv = KFold(10, shuffle=True, random_state=1)
for i in range(1, 11):
    poly = PolynomialFeatures(i).fit(X_train)
    X_train_poly = poly.transform(X_train)
    scores = cross_val_score(lm, X_train_poly, y_train, cv=cv, scoring='neg_mean_squared_error')
    mse_mean = np.mean(-scores)
    mse_std = np.std(-scores) 
    print(f'poly: {i} - mse: {mse_mean}, std: {mse_std}')

poly: 1 - mse: 24.21965632497301, std: 5.655569942971911
poly: 2 - mse: 19.34905165299215, std: 6.75998179990019
poly: 3 - mse: 19.416177107084287, std: 6.847547439455138
poly: 4 - mse: 19.440516673950192, std: 6.793107079879531
poly: 5 - mse: 18.981042354511565, std: 6.7913104674803115
poly: 6 - mse: 18.82446367095536, std: 6.759783670485956
poly: 7 - mse: 19.046671259975064, std: 6.661939491567656
poly: 8 - mse: 19.223943478459454, std: 6.676990786105731
poly: 9 - mse: 19.200980451554507, std: 6.778080978380523
poly: 10 - mse: 19.020593973979583, std: 6.798737088198012
